In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
weather = pd.read_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/measurements.csv")
harvest = pd.read_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/br_final.csv")

In [31]:
minas_harvest = []
harvest["year"]
harvest["bear_mill_trees"]
harvest["nonbear_mill_trees"]
harvest["avg_unemp_perc"]
for place in harvest:
    if harvest["subdivison"] == "Minas Girith":
        minas_harvest.append(harvest.loc())
#"million_60kgs_bag", "nonbear_mill_trees", "bear_mill_trees", "avg_unemp_perc"
print(minas_harvest)

KeyError: 'subdivison'

In [3]:
weather1 = weather.set_index(pd.DatetimeIndex(weather["time"]))

In [14]:
monthly_temps = weather1.resample("ME")["temperature"].median()
#monthly_temps = monthly_temps.drop("time")
yearly_temps = weather1.resample("YE")["temperature"].median()


In [13]:
monthly_humidity = weather1.resample("ME")["humidity"].median()
#monthly_humidity = monthly_humidity.drop("time")
yearly_humidity = weather1.resample("YE")["humidity"].median()

In [10]:
monthly_rain = weather1.resample("ME")["precipitation"].median()
#monthly_rain = monthly_rain.drop("time")
yearly_rain = weather1.resample("YE")["precipitation"].median()


In [12]:
monthly_psr = weather1.resample("ME")["pressure"].median()
#monthly_psr = monthly_psr.drop("time")
yearly_psr = weather1.resample("YE")["pressure"].median()
#df.to_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/YearPressure.csv", index=False)

In [18]:
Monthly_Medians = pd.DataFrame({"temperature":[monthly_temps], "humidity":[monthly_humidity], "precipitation":[monthly_rain], "pressure":[monthly_psr]})
Monthly_Medians.to_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/MonthlyMedians.csv", index=False)
Yearly_Medians =pd.DataFrame({"temperature":[yearly_temps], "humidity":[yearly_humidity], "precipitation":[yearly_rain], "pressure":[yearly_psr]})
Yearly_Medians.to_csv("/Users/sa12/Documents/Repositories/WeatherDataMG/weather_api/data/csv/YearlyMedians.csv", index=False)
